#### 1. 搭建的神经网络，使用olivettiface数据集进行训练


In [ ]:
import torch
import torch.nn as nn
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split



# 定义超参数
LR = 1e-3
epochs = 1000 #训练轮次
BATCH_SIZE = 10  #每批10张


# 1.生成随机样本
X, y = fetch_olivetti_faces(data_home='./face_data',return_X_y=True,shuffle=True)

# #拆分数据
# #局部样本训练模型（过拟合模型）
# #新样板数据模型表现不好（泛化能力差）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


train_data_set = [(img,lbl) for img,lbl in zip(X_train, y_train)]
test_data_set = [(img,lbl) for img,lbl in zip(X_test, y_test)]


train_dl = torch.utils.data.DataLoader(train_data_set, batch_size=BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_data_set, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
# 模型
model = nn.Sequential(
    nn.Linear(4096, 256),  #输入层
    nn.ReLU(),     #隐藏层
    nn.Linear(256, 40)  #输出层
)

#损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数

#优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR) #SGD优化器


for epoch in range(epochs):
    for data, target in train_dl:
        #前向运算
        output = model(data.reshape(-1, 4096))
        #计算损失
        loss = loss_fn(output, target) #计算梯度
        #反向传播
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()  #计算梯度（参数.grad）
        optimizer.step() #更新参数
    print(f'Epoch:{epoch}, Loss:{loss.item()}')

In [ ]:
# 测试
correct = 0
total = 0

model.eval()  #train(false)
with torch.no_grad(): #不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 4096))
        _, predicted = torch.max(output, 1) #返回第一个维度的最大值张量
        total += target.size(0) #返回张量的样本量
        correct += (predicted == target).sum().item()
print(f'Accuracy: {correct/total*100}%')
